In [1]:
import pandas as pd
import numpy as np

# Data Cleaning and Feature Engineering

In this notebook, I use the CSV files we exported from the _Data Collection_ notebook for further cleaning and feature engineering. As a reminder, the features I'm trying to extract are the following:

**Offensive Efficiency:**

-   **Points per game:** Scoring consistently at a high level wins games. Top contenders typically average over 110 points per game.
-   **Field goal percentage:** Converting shots efficiently maximizes opportunities. Look for teams shooting above 45% overall and 35% from three-point range.
-   **Assist-to-turnover ratio:** Moving the ball and taking care of it reduces wasted possessions and creates better scoring chances. Aim for ratios above 2.0.

**Defensive Prowess:**

-   **Points allowed per game:** Limiting the opponent's scoring keeps you in contention. Strong teams hold opponents below 105 points per game.
-   **Rebounds per game:** Controlling the boards limits second-chance opportunities and fuels transition offense. Look for teams grabbing over 50 rebounds per game.

**Balance and Depth:**

-   **Scoring distribution:** Multiple players contributing offensively prevents opponents from focusing on single-stop strategies. Balanced teams have at least 3 players averaging double-digit points.
-   **Bench production:** A reliable bench unit can provide scoring, defensive intensity, and rest for starters. Top teams see significant contributions from their reserves.

**Intangibles:**

-   **Home court advantage:** Winning at home is crucial for playoff seeding and momentum. Strong teams typically boast a winning home record.

### Load the dataframes

In [2]:
team_year_by_year = pd.read_csv('./datasets/result_all_teams.csv')
league_standings = pd.read_csv('./datasets/standings_selected_feat.csv')
rosters_players_combined = pd.read_csv('./datasets/rosters_players_combined.csv')

### Feature engineering (Home Court Advantage)

The home court results are originally formmated as a string with numbers representing 'win-loss'; '24-18', for example. The code below converts it to a winning percentage. In addition, it creates a boolean that represents a team classifying for the playoffs. Finally we rename some columns to match other dataframes.

In [3]:
# Splits the column into two using the '-' separator.
league_standings[['HOME_W', 'HOME_L']] = league_standings['HOME'].str.split('-', expand=True)

# Converts to integer
league_standings['HOME_W'] = league_standings['HOME_W'].astype(int)
league_standings['HOME_L'] = league_standings['HOME_L'].astype(int)

# Creates a calculated column for the win/loss percentage and drop old columns
league_standings['HOME_WIN_PCT'] = league_standings['HOME_W']/(league_standings['HOME_W']+league_standings['HOME_L'])
league_standings = league_standings.drop(['HOME', 'HOME_W', 'HOME_L'], axis=1)

# Renamme columns to match the ones in the other dataframes
league_standings = league_standings.rename(columns={'SeasonID':'YEAR', 'TeamID':'TEAM_ID', 'OppPointsPG':'OPP_PPG'})

# Creates a boolean variable indicating if a team classified to the playoffs that year
league_standings['PLAYOFF'] = np.where(league_standings['PlayoffRank']<=8, 1, 0)
league_standings = league_standings.drop(['PlayoffRank'], axis=1)

In [4]:
league_standings

,YEAR,TEAM_ID,TeamName,OPP_PPG,HOME_WIN_PCT,PLAYOFF
0,1984,1610612738,Celtics,108.1,0.853659,1
1,1984,1610612747,Lakers,110.9,0.878049,1
2,1984,1610612743,Nuggets,117.6,0.829268,1
3,1984,1610612749,Bucks,104.0,0.878049,1
4,1984,1610612745,Rockets,109.5,0.707317,1
...,...,...,...,...,...,...
1105,2022,1610612757,Trail Blazers,117.4,0.414634,0
1106,2022,1610612745,Rockets,118.6,0.341463,0
1107,2022,1610612766,Hornets,117.2,0.317073,0
1108,2022,1610612759,Spurs,123.1,0.341463,0


Merge `league_standings` dataframe with the `team_year_by_year` dataframe resulting in a dataframe with all the data concerning team stats.

In [5]:
all_teams_data = team_year_by_year.merge(league_standings, on=['TEAM_ID', 'YEAR'], how='inner')
all_teams_data = all_teams_data.drop(['TeamName'], axis=1)

In [6]:
all_teams_data

,TEAM_ID,TEAM_NAME,YEAR,WIN_PCT,FG_PCT,REB,AST_TOV_RAT,PTS,OPP_PPG,HOME_WIN_PCT,PLAYOFF
0,1610612739,Cavaliers,1984,0.439,0.471,44.5,1.514793,108.6,111.3,0.487805,1
1,1610612739,Cavaliers,1985,0.354,0.480,43.2,1.465116,107.8,110.6,0.390244,0
2,1610612739,Cavaliers,1986,0.378,0.470,44.8,1.182741,104.4,108.2,0.609756,0
3,1610612739,Cavaliers,1987,0.512,0.490,40.3,1.440000,104.5,103.7,0.756098,1
4,1610612739,Cavaliers,1988,0.695,0.502,42.8,1.714286,108.8,101.2,0.902439,1
...,...,...,...,...,...,...,...,...,...,...,...
1105,1610612755,76ers,2018,0.622,0.471,47.8,1.805369,115.2,112.5,0.756098,1
1106,1610612755,76ers,2019,0.589,0.468,45.4,1.816901,110.7,108.4,0.885714,1
1107,1610612755,76ers,2020,0.681,0.476,45.1,1.645833,113.6,108.1,0.805556,1
1108,1610612755,76ers,2021,0.622,0.466,42.3,1.896000,109.9,107.3,0.585366,1


### Feature Selection Rosters and Players

In [7]:
rosters_players_selected = rosters_players_combined.drop(['LeagueID', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'HOW_ACQUIRED', 'LEAGUE_ID', 'PLAYER_AGE'], axis=1)
rosters_players_selected['PPG'] = rosters_players_selected['PTS']/rosters_players_selected['GP']  
rosters_players_selected['MINPG'] = rosters_players_selected['MIN']/rosters_players_selected['GP']

In [8]:
rosters_players_selected

,TEAM_ID,SEASON_ID,PLAYER,POSITION,PLAYER_ID,TEAM_ABBREVIATION,GP,GS,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PPG,MINPG
0,1610612738,1984,Danny Ainge,G,76017,BOS,75,73.0,2564.0,419,...,192.0,268,399,122.0,6.0,149.0,228,971,12.946667,34.186667
1,1610612738,1984,Larry Bird,F,1449,BOS,80,77.0,3161.0,918,...,678.0,842,531,129.0,98.0,248.0,208,2295,28.687500,39.512500
2,1610612738,1984,Quinn Buckner,G,76296,BOS,75,6.0,858.0,74,...,61.0,87,148,63.0,2.0,67.0,142,180,2.400000,11.440000
3,1610612738,1984,Rick Carlisle,G,76342,BOS,38,0.0,179.0,26,...,13.0,21,25,3.0,0.0,19.0,21,67,1.763158,4.710526
4,1610612738,1984,M.L. Carr,F,76351,BOS,47,0.0,397.0,62,...,22.0,43,24,21.0,6.0,24.0,44,150,3.191489,8.446809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15784,1610612740,2022,Jose Alvarado,G,1630631,NOP,61,10.0,1310.0,201,...,113.0,141,186,67.0,10.0,81.0,125,550,9.016393,21.475410
15785,1610612740,2022,Jonas Valanciunas,C,202685,NOP,79,79.0,1968.0,446,...,579.0,804,140,20.0,52.0,159.0,245,1115,14.113924,24.911392
15786,1610612740,2022,Larry Nance Jr.,F-C,1626204,NOP,65,1.0,1381.0,186,...,248.0,354,119,56.0,37.0,42.0,133,441,6.784615,21.246154
15787,1610612740,2022,Trey Murphy III,F,1630530,NOP,79,65.0,2448.0,387,...,222.0,281,112,89.0,43.0,61.0,155,1148,14.531646,30.987342


### Feature Engineering (Scoring Distribution)

-   **Scoring distribution:** Multiple players contributing offensively prevents opponents from focusing on single-stop strategies. Balanced teams have at least 3 players averaging double-digit points.

For this feature, I created a dataframe with only the top 3 scorers for each team and season using their average points per game.

In [9]:
# Group by Team and Year, order by greatest points per game average, and keep only the top 3 in each group
rosters_players_top3 = rosters_players_selected.sort_values('PTS', ascending=False).groupby(["TEAM_ID", "SEASON_ID"]).head(3)
rosters_players_top3 = rosters_players_top3.sort_values(['TEAM_ID','SEASON_ID']).reset_index(drop=True)

Next, I created a dataframe with only the lowest scorer of each group in `roster_players_top3` to check whether he scored at least 10 points.

In [11]:
# Extracted only the lowest scorer out of the top 3
rosters_players_top3_min = rosters_players_top3[['TEAM_ID', 'SEASON_ID', 'PPG']].groupby(by=['TEAM_ID', 'SEASON_ID']).min().reset_index()

# Created boolean to represent teams and years where all the top 3 scorers averaged at least 10 points
rosters_players_top3_min['TOP3_P14'] = np.where(rosters_players_top3_min['PPG']>=10, 1, 0)
rosters_players_top3_min = rosters_players_top3_min.rename(columns={'SEASON_ID':'YEAR'})
rosters_players_top3_min = rosters_players_top3_min.drop(['PPG'], axis=1)

In [12]:
rosters_players_top3_min

,TEAM_ID,YEAR,TOP3_P14
0,1610612737,1984,1
1,1610612737,1985,1
2,1610612737,1986,1
3,1610612737,1987,1
4,1610612737,1988,1
...,...,...,...
1105,1610612766,2018,1
1106,1610612766,2019,1
1107,1610612766,2020,1
1108,1610612766,2021,1


Finally, I created an additional column with the average points per game for the top 3 players for each team and year.

In [13]:
rosters_players_top3_mean = rosters_players_top3[['TEAM_ID', 'SEASON_ID', 'PPG']].groupby(by=['TEAM_ID', 'SEASON_ID']).mean().reset_index()
rosters_players_top3_mean = rosters_players_top3_mean.rename(columns={'PPG':'TOP3_PPG', 'SEASON_ID':'YEAR'})

In [14]:
rosters_players_top3_mean

,TEAM_ID,YEAR,TOP3_PPG
0,1610612737,1984,19.276259
1,1610612737,1985,18.508983
2,1610612737,1986,19.326068
3,1610612737,1987,18.839701
4,1610612737,1988,20.750752
...,...,...,...
1105,1610612766,2018,17.001762
1106,1610612766,2019,16.381278
1107,1610612766,2020,17.577404
1108,1610612766,2021,19.852279


-   **Bench production:** A reliable bench unit can provide scoring, defensive intensity, and rest for starters. Top teams see significant contributions from their reserves.

To extract the bench players from the players and rosters dataset, I first identified the top 5 players who played the most minutes and created the `roster_players_starters` dataframe. I assumed all other players to be 'bench' players. Next, I subtracted the original dataset with the `roster_players_starters` to create `off_the_bench`, that contained only bench players.

In [15]:
# Extracts the top 5 players with most minutes played
rosters_players_starters = rosters_players_selected.sort_values('MIN', ascending=False).groupby(["TEAM_ID", "SEASON_ID"]).head(5)
rosters_players_starters = rosters_players_starters.sort_values(['TEAM_ID','SEASON_ID'])

#extracted the indexes to subtract the starters dataframe from the original dataframe
starters_idx = list(rosters_players_starters.index.values)
off_the_bench = rosters_players_selected.drop(index=starters_idx)

Next, I grouped the `off_the_bench` dataframe by team and year and added the total points of all bench players for each team and year. Since there are 82 games per season, I divided the total points by 82 to get the average points per game by bench players. Finally, I created a boolean to represent whether a team averages at least 30 points per game "off the bench".

In [17]:
# Sums the total points for all players and divide by 82
off_the_bench_sum = off_the_bench[['TEAM_ID', 'SEASON_ID', 'PTS']].groupby(by=['TEAM_ID', 'SEASON_ID']).sum().reset_index()
off_the_bench_sum['OFF_BENCH_PPG'] = off_the_bench_sum['PTS']/82

# Creates a boolean for wheter a team averages at least 30 points off the bench for a given year
off_the_bench_sum['BENCH_P30'] = np.where(off_the_bench_sum['OFF_BENCH_PPG']>=30, 1, 0)
off_the_bench_sum = off_the_bench_sum.rename(columns={'SEASON_ID':'YEAR'})
off_the_bench_sum = off_the_bench_sum.drop('PTS', axis=1)

In [18]:
off_the_bench_sum

,TEAM_ID,YEAR,OFF_BENCH_PPG,BENCH_P30
0,1610612737,1984,34.963415,1
1,1610612737,1985,40.426829,1
2,1610612737,1986,34.280488,1
3,1610612737,1987,33.524390,1
4,1610612737,1988,28.097561,0
...,...,...,...,...
1105,1610612766,2018,45.353659,1
1106,1610612766,2019,23.024390,0
1107,1610612766,2020,38.182927,1
1108,1610612766,2021,37.451220,1


## Merging Data from Rosters and Players with Standings Data

In [19]:
all_teams_full = all_teams_data.copy()
all_teams_full = all_teams_full.merge(rosters_players_top3_mean, how='inner', on=['TEAM_ID', 'YEAR'])
all_teams_full = all_teams_full.merge(rosters_players_top3_min, how='inner', on=['TEAM_ID', 'YEAR'])
all_teams_full = all_teams_full.merge(off_the_bench_sum, how='inner', on=['TEAM_ID', 'YEAR'])

In [20]:
all_teams_full

,TEAM_ID,TEAM_NAME,YEAR,WIN_PCT,FG_PCT,REB,AST_TOV_RAT,PTS,OPP_PPG,HOME_WIN_PCT,PLAYOFF,TOP3_PPG,TOP3_P14,OFF_BENCH_PPG,BENCH_P30
0,1610612739,Cavaliers,1984,0.439,0.471,44.5,1.514793,108.6,111.3,0.487805,1,18.032740,1,34.951220,1
1,1610612739,Cavaliers,1985,0.354,0.480,43.2,1.465116,107.8,110.6,0.390244,0,18.894844,1,28.853659,0
2,1610612739,Cavaliers,1986,0.378,0.470,44.8,1.182741,104.4,108.2,0.609756,0,17.705386,1,31.109756,1
3,1610612739,Cavaliers,1987,0.512,0.490,40.3,1.440000,104.5,103.7,0.756098,1,16.714243,1,27.926829,0
4,1610612739,Cavaliers,1988,0.695,0.502,42.8,1.714286,108.8,101.2,0.902439,1,18.791115,1,28.060976,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,1610612755,76ers,2018,0.622,0.471,47.8,1.805369,115.2,112.5,0.756098,1,20.830769,1,23.475610,0
1106,1610612755,76ers,2019,0.589,0.468,45.4,1.816901,110.7,108.4,0.885714,1,19.672070,1,37.109756,1
1107,1610612755,76ers,2020,0.681,0.476,45.1,1.645833,113.6,108.1,0.805556,1,20.753404,1,38.341463,1
1108,1610612755,76ers,2021,0.622,0.466,42.3,1.896000,109.9,107.3,0.585366,1,21.757569,1,28.048780,0


In [21]:
all_teams_full.to_csv('all_teams_full_cleaned.csv', index=False)